# Capstone Project - Short Answer Grading

## SciEntsBank Data and Modeling

In [ ]:
# source code for SKLearn custom train.py
!pygmentize source/train.py

# Setup AWS

In [ ]:
import pandas as pd
import boto3
import sagemaker

# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

In [ ]:
# should be the name of directory you created to save your features data
data_dir = 'data/sag2'

# set prefix, a descriptive name for a directory  
prefix = 'sagemaker/short_answer'

# upload all data to S3
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

## Analyzing Source Data

# Training Basic Model

In [ ]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='source/train.py', 
                       role=role,
                       train_instance_count=1, 
                       train_instance_type='ml.c4.xlarge',
                       framework_version='1.12.0', 
                       py_version='py3',
                       script_mode=True,
                       hyperparameters = {
                        'epochs': 200,
                        'embedding_size': 50,
                        'flatten': 1,   
                        'lstm_dim_1': 100,
                        'lstm_dim_2': 20,
                        'dropout': 0.2
                    })

In [ ]:
%%time
# Train estimator on S3 training data
estimator.fit(input_data)

# Hypertuning Model

In [ ]:
# First, make sure to import the relevant objects used to construct the tuner
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

tf_hyperparameter_tuner = HyperparameterTuner(estimator = estimator, # The estimator object to use as the basis for the training jobs.
                                               objective_metric_name = 'Validation_accuracy', # The metric used to compare trained models.
                                               objective_type = 'Maximize', # Whether we wish to minimize or maximize the metric.
                                               metric_definitions = [{'Name': 'Validation_loss', 
                                                                      'Regex': 'Validation_loss:(.*?);'},
                                                                     {'Name': 'Validation_accuracy', 
                                                                      'Regex': 'Validation_accuracy:(.*?);'}
                                                                    ],
                                               max_jobs = 18, # The total number of models to train
                                               max_parallel_jobs = 6, # The number of models to train in parallel
                                               hyperparameter_ranges = {
                                                    'dropout': ContinuousParameter(0.1, 0.3),
                                                    'embedding_size': IntegerParameter(50, 200),
                                                    'lstm_dim_1': IntegerParameter(50, 200),
                                                    'lstm_dim_2': IntegerParameter(10, 50)
                                               })


In [ ]:
# tf_hyperparameter_tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})
tf_hyperparameter_tuner.fit(input_data)

In [10]:
tf_hyperparameter_tuner.wait()

...........................................................................................................................................................................................................................................................................................................................................................................................................................!


In [11]:
tf_hyperparameter_tuner.best_training_job()

'sagemaker-tensorflow-190630-1620-003-e4783a22'

In [12]:
# Reload an existing trained estimator
training_job_name = tf_hyperparameter_tuner.best_training_job()
estimator = TensorFlow.attach(training_job_name)

2019-06-30 16:28:57 Starting - Preparing the instances for training
2019-06-30 16:28:57 Downloading - Downloading input data
2019-06-30 16:28:57 Training - Training image download completed. Training in progress.
2019-06-30 16:28:57 Uploading - Uploading generated training model
2019-06-30 16:28:57 Completed - Training job completed2019-06-30 16:23:31,685 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2019-06-30 16:23:31,686 sagemaker-containers INFO     Failed to parse hyperparameter _tuning_objective_metric value Validation_accuracy to Json.
Returning the value itself
2019-06-30 16:23:31,691 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-06-30 16:23:31,704 sagemaker_tensorflow_container.training INFO     Appending the training job name to model_dir: s3://sagemaker-us-east-1-399712746635/sagemaker-tensorflow-scriptmode-2019-06-30-16-14-42-271/model/sagemaker-tensorflow-190630-1620-003-e4783a22/model
2019-06-3

In [13]:
predictor = estimator.deploy(initial_instance_count=1,
                             # instance_type='ml.c5.xlarge',
                             instance_type='ml.t2.medium',
                             endpoint_type='tensorflow-serving')

--------------------------------------------------------------------------------------!

## Load and Evaluate Test Data

In [ ]:
import numpy as np
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # rounding and squeezing array
    test_preds = np.squeeze(predictor.predict(test_features)['predictions'])
    # Normalized to range 0 to 1
    min_red = test_y_preds.min()
    max_pred = test_y_preds.max()
    test_y_preds = (test_y_preds - min_pred) / (max_pred - min_pred)
    test_preds = np.round(test_preds)    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # print metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actuals'], colnames=['predictions']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print()
        
    return tp, fp, fn, tn, precision, recall, accuracy

In [ ]:
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)
test_y = test_data.iloc[:, 0].values
test_x = test_data.iloc[:, 1:].values

In [ ]:
# First: generate predicted, class labels
test_y_preds = np.squeeze(predictor.predict(test_x)['predictions'])
#normalize to 0 - 1
min_red = test_y_preds.min()
max_pred = test_y_preds.max()
test_y_preds = (test_y_preds - min_pred) / (max_pred - min_pred)
print(test_y_preds)    

In [ ]:
# Second: calculate the test accuracy
tp, fp, fn, tn, precision, recall, accuracy = evaluate(predictor, test_x, test_y)

## print out the array of predicted and true labels, if you want
results = pd.concat([pd.DataFrame(test_y_preds), pd.DataFrame(np.round(test_y_preds)), pd.DataFrame(test_y)], axis=1)
results.columns = ['raw_predicted','predicted','actual']
incorrect_results = results[results['predicted'] != results['actual']]
print(incorrect_results)